In [4]:
!git clone https://github.com/gbrltv/process_meta_learning.git

Cloning into 'process_meta_learning'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1155 (delta 20), reused 38 (delta 12), pack-reused 1101
Receiving objects: 100% (1155/1155), 117.74 MiB | 33.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [6]:
import pandas as pd

df = pd.read_csv("process_meta_learning/results/log_features.csv")
df

,log,n_traces,n_unique_traces,ratio_unique_traces_per_trace,n_events,trace_len_min,trace_len_max,trace_len_mean,trace_len_median,trace_len_mode,trace_len_std,trace_len_variance,trace_len_q1,trace_len_q3,trace_len_iqr,trace_len_geometric_mean,trace_len_geometric_std,trace_len_harmonic_mean,trace_len_skewness,trace_len_kurtosis,trace_len_coefficient_variation,trace_len_entropy,trace_len_hist1,trace_len_hist2,trace_len_hist3,trace_len_hist4,trace_len_hist5,trace_len_hist6,trace_len_hist7,trace_len_hist8,trace_len_hist9,trace_len_hist10,trace_len_skewness_hist,trace_len_kurtosis_hist,ratio_most_common_variant,ratio_top_1_variants,ratio_top_5_variants,ratio_top_10_variants,ratio_top_20_variants,ratio_top_50_variants,...,activities_iqr,activities_skewness,activities_kurtosis,n_unique_start_activities,start_activities_min,start_activities_max,start_activities_mean,start_activities_median,start_activities_std,start_activities_variance,start_activities_q1,start_activities_q3,start_activities_iqr,start_activities_skewness,start_activities_kurtosis,n_unique_end_activities,end_activities_min,end_activities_max,end_activities_mean,end_activities_median,end_activities_std,end_activities_variance,end_activities_q1,end_activities_q3,end_activities_iqr,end_activities_skewness,end_activities_kurtosis,entropy_trace,entropy_prefix,entropy_global_block,entropy_lempel_ziv,entropy_k_block_diff_1,entropy_k_block_diff_3,entropy_k_block_diff_5,entropy_k_block_ratio_1,entropy_k_block_ratio_3,entropy_k_block_ratio_5,entropy_knn_3,entropy_knn_5,entropy_knn_7
0,bpi_2012,13087,4379,0.334607,262200,3,175,20.035149,11.0,3,19.943559,397.745536,3.0,31.0,28.0,11.807100,2.934917,7.124952,1.618213,3.876497,0.995429,9.042624,0.034732,0.013283,0.006535,0.002279,0.000849,0.000244,0.000120,0.000058,0.000013,0.000027,2.054982,2.926423,0.262016,0.560938,0.644151,0.685489,0.732253,0.832658,...,10841.00,2.144346,3.692864,1,13087,13087,13087.000000,13087.0,0.000000,0.000000e+00,13087.0,13087.00,0.00,0.000000,-3.000000,11,1,3429,1189.727273,653.0,1162.557767,1.351541e+06,168.0,2087.00,1919.00,0.604934,-1.048214,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,bpi_2012_A,13087,17,0.001299,60849,3,8,4.649576,4.0,3,1.882808,3.544965,3.0,6.0,3.0,4.309628,1.464974,4.028236,0.731763,-0.966721,0.404942,9.401915,0.884695,0.000000,0.334378,0.000000,0.064491,0.000000,0.373195,0.000000,0.000000,0.343241,1.391334,1.022865,0.436999,0.000000,0.000000,0.436999,0.646367,0.905937,...,5181.75,0.764459,-0.794840,1,13087,13087,13087.000000,13087.0,0.000000,0.000000e+00,13087.0,13087.00,0.00,0.000000,-3.000000,8,3,7635,1635.875000,630.0,2416.367751,5.838833e+06,262.5,1441.25,1178.75,1.812261,1.824614,2.838,3.452,5.839,0.008,3.027,3.027,3.027,3.027,3.027,3.027,1.151,0.727,0.719
2,bpi_2012_O,5015,168,0.033500,31244,3,30,6.230110,5.0,5,3.169158,10.043560,4.0,8.0,4.0,5.682570,1.491033,5.304181,2.281176,6.899116,0.508684,8.415881,0.264170,0.029246,0.047266,0.017208,0.006204,0.003176,0.001920,0.000443,0.000517,0.000222,2.494657,4.542380,0.306481,0.306481,0.798804,0.891326,0.946760,0.983051,...,4181.50,-0.114659,-1.514451,1,5015,5015,5015.000000,5015.0,0.000000,0.000000e+00,5015.0,5015.00,0.00,0.000000,-3.000000,5,89,2243,1003.000000,802.0,824.598084,6.799620e+05,241.0,1640.00,1399.00,0.339213,-1.457644,3.753,4.535,6.981,0.107,-0.273,0.426,0.285,0.769,2.574,1.500,2.740,2.392,2.208
3,bpi_2012_W,9658,2643,0.273659,170107,2,156,17.613067,14.0,3,15.986745,255.576002,5.0,25.0,20.0,11.685828,2.592474,7.596637,2.002149,7.226360,0.907664,8.823251,0.038888,0.016694,0.006354,0.001903,0.000585,0.000296,0.000087,0.000067,0.000020,0.000040,1.989750,2.614140,0.193829,0.445434,0.620522,0.701077,0.772106,0.863119,...,29988.00,0.364298,-1.258906,3,67,4852,3219.333333,4739.0,2229.513599,4.970731e+06,2403.0,4795.50,2392.50,-0.705745,-1.500000,7,4,2749,1379.714286,1802.0,1084.394197,1.175911e+06,256.5,2295.00,2038.50,-0.169172,-1.723262,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,